## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Apr. 29th May. 6th


In [3]:
filename = glob.glob('redfin*.csv')
filename = filename[0]

In [4]:
df = pd.read_csv(filename)
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [6]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

179


In [7]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$635,738


In [8]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$529


In [9]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$113,797,141


## Previous Week's Values

In [10]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
155
Input Previous Week Condo Average Sales Price:
789242
Input Previous Week Condo Average PSF:
545
Input Previous Week Condo Sales total (ex: 198_000_000)
122000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [11]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [12]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
pd.set_option('display.max_columns',None)

In [15]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
175,PAST SALE,April-29-2024,Condo/Co-op,321 Ocean Dr #300,Miami Beach,FL,33139.0,7250000.0,4.0,4.5,321 OCEAN CONDO,3227.0,NaN,2015.0,NaN,2247.0,9211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11393005,N,Y,25.772296,-80.131901,1,orange


In [16]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [17]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [18]:
m.save('index.html')

## Data snagger

In [19]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [20]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
175,PAST SALE,April-29-2024,Condo/Co-op,321 Ocean Dr #300,Miami Beach,FL,33139.0,7250000.0,4.0,4.5,321 OCEAN CONDO,3227.0,NaN,2015.0,NaN,2247.0,9211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11393005,N,Y,25.772296,-80.131901,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [21]:
df_top_ten['PRICE'].iloc[0]

7250000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [22]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_050624


# CREATE TEMPLATE 

In [23]:
muni_set = set(df_top_ten['CITY'])

In [24]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [25]:
df_top_ten.reset_index(inplace=True,drop=True)

In [26]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [27]:
top_sale

'321 OCEAN CONDO at 321 Ocean Dr #300 in Miami Beach'

In [29]:
# previous_week_condo_sales_total = 130_000_000

In [28]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [29]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 321 OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,610,000M to $7,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bay Harbor Islands, Miami Beach, Miami, Bal Harbour, Key Biscayne, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 179 condo sales totaling $113,797,141 million from Apr. 29th to May. 6th. The previous week, brokers closed 155 condo sales totaling $122,000,000.

Last week’s units sold for an average of $635,738, lower than the $789,242 sales average from the pr

In [30]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [32]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: 321 OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,610,000M to $7,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bay Harbor Islands, Miami Beach, Miami, Bal Harbour, Key Biscayne, Sunny Isles Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 179 condo sales totaling $113,797,141 million from Apr. 29th to May. 6th. The previous week, brokers closed 155 condo sales totaling $122,000,000.

Last week’s units sold for an average of $635,738, lower than the $789,242 sales average from the pr

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,April-29-2024,Condo/Co-op,321 Ocean Dr #300,Miami Beach,FL,33139.0,7250000.0,4.0,4.5,321 OCEAN CONDO,3227.0,NaN,2015.0,NaN,2247.0,9211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11393005,N,Y,25.772296,-80.131901,1,orange,321 OCEAN CONDO at 321 Ocean Dr #300 in Miami ...
1,PAST SALE,May-1-2024,Condo/Co-op,2901 S Bayshore Dr Unit 9BC,Miami,FL,33133.0,3500000.0,5.0,3.5,YACHT HARBOUR,3174.0,NaN,1975.0,NaN,1103.0,3780.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11439999,N,Y,25.727753,-80.239494,2,blue,YACHT HARBOUR at 2901 S Bayshore Dr Unit 9BC i...
2,PAST SALE,April-29-2024,Condo/Co-op,10295 Collins Ave #405,Bal Harbour,FL,33154.0,3273000.0,2.0,3.0,10295 COLLINS AVE RESDNTA,2221.0,NaN,2007.0,NaN,1474.0,3649.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11474946,N,Y,25.898952,-80.123405,3,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
3,PAST SALE,April-30-2024,Condo/Co-op,1445 16th St Unit UP-3,Miami Beach,FL,33139.0,2500000.0,2.0,3.5,CAPRI SOUTH BEACH CONDO,1919.0,NaN,2008.0,NaN,1303.0,2794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11408686,N,Y,25.789262,-80.144721,4,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Unit U...
4,PAST SALE,April-30-2024,Condo/Co-op,17001 Collins Ave #2004,Sunny Isles Beach,FL,33160.0,2460000.0,3.0,3.5,JADE BEACH,1927.0,NaN,2008.0,NaN,1277.0,2237.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11492768,N,Y,25.933532,-80.120953,5,blue,JADE BEACH at 17001 Collins Ave #2004 in Sunny...
5,PAST SALE,April-30-2024,Condo/Co-op,400 S Pointe Dr #1404,Miami Beach,FL,33139.0,2250000.0,3.0,3.0,South Pointe Tower,1802.0,NaN,1987.0,NaN,1249.0,3028.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11410961,N,Y,25.767827,-80.134241,6,blue,South Pointe Tower at 400 S Pointe Dr #1404 in...
6,PAST SALE,May-1-2024,Condo/Co-op,2900 NE 7th Ave #2807,Miami,FL,33137.0,2200000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,1178.0,2337.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11475293,N,Y,25.805017,-80.185768,7,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #2807 ...
7,PAST SALE,April-30-2024,Condo/Co-op,1111 Crandon Blvd Unit A1202,Key Biscayne,FL,33149.0,2020000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,1134.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11502248,N,Y,25.678856,-80.167404,8,blue,TOWERS OF KEY BISCAYNE CO at 1111 Crandon Blvd...
8,PAST SALE,April-29-2024,Condo/Co-op,18201 Collins Ave #3709,Sunny Isles Beach,FL,33160.0,1750000.0,2.0,2.5,TRUMP ROYALE CONDO,1722.0,NaN,2008.0,NaN,1016.0,1814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11536953,N,Y,25.944990,-80.120089,9,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #3709 ...
9,PAST SALE,May-2-2024,Condo/Co-op,10350 W Bay Harbor Dr Unit 10RS,Bay Harbor Islands,FL,33154.0,1610000.0,4.0,3.0,ISLAND POINTE CONDO,2330.0,NaN,1966.0,NaN,691.0,5048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10350-W-...,MARMLS,A11534686,N,Y,25.893940,-80.133398,10,blue,ISLAND POINTE CONDO at 10350 W Bay Harbor Dr U...


In [34]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/321-Ocean-Dr-33139/unit-300/home/45471084


In [44]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/10350-W-Bay-Harbor-Dr-33154/unit-10RS/home/189771212


In [36]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/18201-Collins-Ave-33160/unit-3709/home/43406675


In [37]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/321-Ocean-Dr-33139/unit-300/home/45471084


In [38]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami-Beach/10350-W-Bay-Harbor-Dr-33154/unit-10RS/home/189771212


In [39]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,April-29-2024,Condo/Co-op,321 Ocean Dr #300,Miami Beach,FL,33139.0,7250000.0,4.0,4.5,321 OCEAN CONDO,3227.0,NaN,2015.0,NaN,2247.0,9211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/321-Ocea...,MARMLS,A11393005,N,Y,25.772296,-80.131901,1,orange,321 OCEAN CONDO at 321 Ocean Dr #300 in Miami ...
1,PAST SALE,May-1-2024,Condo/Co-op,2901 S Bayshore Dr Unit 9BC,Miami,FL,33133.0,3500000.0,5.0,3.5,YACHT HARBOUR,3174.0,NaN,1975.0,NaN,1103.0,3780.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11439999,N,Y,25.727753,-80.239494,2,blue,YACHT HARBOUR at 2901 S Bayshore Dr Unit 9BC i...
2,PAST SALE,April-29-2024,Condo/Co-op,10295 Collins Ave #405,Bal Harbour,FL,33154.0,3273000.0,2.0,3.0,10295 COLLINS AVE RESDNTA,2221.0,NaN,2007.0,NaN,1474.0,3649.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10295-Co...,MARMLS,A11474946,N,Y,25.898952,-80.123405,3,blue,10295 COLLINS AVE RESDNTA at 10295 Collins Ave...
3,PAST SALE,April-30-2024,Condo/Co-op,1445 16th St Unit UP-3,Miami Beach,FL,33139.0,2500000.0,2.0,3.5,CAPRI SOUTH BEACH CONDO,1919.0,NaN,2008.0,NaN,1303.0,2794.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11408686,N,Y,25.789262,-80.144721,4,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Unit U...
4,PAST SALE,April-30-2024,Condo/Co-op,17001 Collins Ave #2004,Sunny Isles Beach,FL,33160.0,2460000.0,3.0,3.5,JADE BEACH,1927.0,NaN,2008.0,NaN,1277.0,2237.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11492768,N,Y,25.933532,-80.120953,5,blue,JADE BEACH at 17001 Collins Ave #2004 in Sunny...
5,PAST SALE,April-30-2024,Condo/Co-op,400 S Pointe Dr #1404,Miami Beach,FL,33139.0,2250000.0,3.0,3.0,South Pointe Tower,1802.0,NaN,1987.0,NaN,1249.0,3028.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/400-S-Po...,MARMLS,A11410961,N,Y,25.767827,-80.134241,6,blue,South Pointe Tower at 400 S Pointe Dr #1404 in...
6,PAST SALE,May-1-2024,Condo/Co-op,2900 NE 7th Ave #2807,Miami,FL,33137.0,2200000.0,3.0,4.0,BISCAYNE BEACH CONDO,1868.0,NaN,2017.0,NaN,1178.0,2337.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2900-NE-7th-Av...,MARMLS,A11475293,N,Y,25.805017,-80.185768,7,blue,BISCAYNE BEACH CONDO at 2900 NE 7th Ave #2807 ...
7,PAST SALE,April-30-2024,Condo/Co-op,1111 Crandon Blvd Unit A1202,Key Biscayne,FL,33149.0,2020000.0,2.0,2.0,TOWERS OF KEY BISCAYNE CO,1782.0,NaN,1972.0,NaN,1134.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/1111-Cr...,MARMLS,A11502248,N,Y,25.678856,-80.167404,8,blue,TOWERS OF KEY BISCAYNE CO at 1111 Crandon Blvd...
8,PAST SALE,April-29-2024,Condo/Co-op,18201 Collins Ave #3709,Sunny Isles Beach,FL,33160.0,1750000.0,2.0,2.5,TRUMP ROYALE CONDO,1722.0,NaN,2008.0,NaN,1016.0,1814.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11536953,N,Y,25.944990,-80.120089,9,blue,TRUMP ROYALE CONDO at 18201 Collins Ave #3709 ...
9,PAST SALE,May-2-2024,Condo/Co-op,10350 W Bay Harbor Dr Unit 10RS,Bay Harbor Islands,FL,33154.0,1610000.0,4.0,3.0,ISLAND POINTE CONDO,2330.0,NaN,1966.0,NaN,691.0,5048.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10350-W-...,MARMLS,A11534686,N,Y,25.893940,-80.133398,10,blue,ISLAND POINTE CONDO at 10350 W Bay Harbor Dr U...


## Time on Market Calculator

In [43]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 8, 22) ## List (Earlier) date
date2 = datetime(2024, 5, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

253
